<a href="https://colab.research.google.com/github/componavt/sns4human/blob/main/src/vk/vk_group_all_post_with_comments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd
import requests
import time
from datetime import datetime
import pytz
from config import TOKEN, VERSION

In [2]:
def get_vk_posts_with_comments(domain, access_token, count=100):
    posts_url = 'https://api.vk.com/method/wall.get'
    comments_url = 'https://api.vk.com/method/wall.getComments'

    posts = []
    offset = 0

    while True:
        response = requests.get(posts_url, params={
            'domain': domain,
            'count': count,
            'offset': offset,
            'access_token': access_token,
            'v': '5.131'
        }).json()

        if 'response' not in response:
            print("Ошибка при получении постов:", response)
            break

        posts_batch = response['response']['items']
        if not posts_batch:
            break

        for post in posts_batch:
            post_id = post['id']
            owner_id = post['owner_id']


            comments_response = requests.get(comments_url, params={
                'owner_id': owner_id,
                'post_id': post_id,
                'access_token': access_token,
                'v': '5.131'
            }).json()

            if 'response' in comments_response:
                comments = comments_response['response'].get('items', [])
                post['comments'] = comments
            else:
                print("Ошибка при получении комментариев:", comments_response)

            posts.append(post)

        offset += count

    return posts


In [3]:
domain='krest_sobor'

In [4]:
posts = get_vk_posts_with_comments(domain, TOKEN)

In [5]:
df = pd.DataFrame(posts)

In [6]:
df.head()

,inner_type,donut,is_pinned,comments,marked_as_ads,hash,type,attachments,copyright,date,...,owner_id,post_type,reposts,text,views,copy_history,edited,carousel_offset,signer_id,zoom_text
0,wall_wallpost,{'is_donut': False},1.0,[],0,NzCNvK0Y6J7nAD3X7w,post,"[{'type': 'photo', 'photo': {'album_id': -7, '...","{'link': 'https://vk.com/wall95551542_37647', ...",1728851421,...,-81948990,post,{'count': 1},РАСПИСАНИЕ БОГОСЛУЖЕНИЙ В КРЕСТОВОЗДВИЖЕНСКОМ ...,{'count': 1261},NaN,NaN,NaN,NaN,NaN
1,wall_wallpost,{'is_donut': False},NaN,[],0,lggWRwObLrwQc82fRw,post,[],NaN,1729227601,...,-81948990,post,{'count': 2},,{'count': 192},"[{'inner_type': 'wall_wallpost', 'type': 'post...",NaN,NaN,NaN,NaN
2,wall_wallpost,{'is_donut': False},NaN,[],0,HBq21UVQQx7GMRwuGQ,post,"[{'type': 'photo', 'photo': {'album_id': 27261...",NaN,1729172061,...,-81948990,post,{'count': 5},"20 октября, в воскресенье, Божественную литург...",{'count': 994},NaN,NaN,NaN,NaN,NaN
3,wall_wallpost,{'is_donut': False},NaN,"[{'id': 13337, 'from_id': 132910578, 'date': 1...",0,BiC7JxDQasKuYe2ZuA,post,[],NaN,1729082164,...,-81948990,post,{'count': 2},,{'count': 158},"[{'inner_type': 'wall_wallpost', 'type': 'post...",NaN,NaN,NaN,NaN
4,wall_wallpost,{'is_donut': False},NaN,[],0,TlHsjDIgikZL_4LoAA,post,"[{'type': 'photo', 'photo': {'album_id': -7, '...",NaN,1729031499,...,-81948990,post,{'count': 5},"Огромное спасибо всем, кто уже поучаствовал в ...",{'count': 819},NaN,1.729066e+09,NaN,NaN,NaN


In [10]:
print(df['comments'].values)

[list([]) list([]) list([]) ...
 list([{'id': 10, 'from_id': 247792480, 'date': 1419022140, 'text': 'Чтобы люди непросвещенные узнали точно обозначенный Крестовоздвиженскийц собор, пометьте его стрелкой. На личном опыте проверено именно с этим фото, что далеко не все с первого раза могут указать правильно собор. Фото и впрямь содержательное: (слева направо) - часовня Николая Чудотворца, Троицкая церковь, Крестовоздвиженский храм, Святодуховский собор, Петропавлоский и Вознесенский соборы. Место съемки: с железной дороги на дороге, ведущей на Вознесенье.', 'post_id': 6, 'owner_id': -81948990, 'parents_stack': [], 'thread': {'count': 0, 'items': [], 'can_post': True, 'show_reply_button': True, 'groups_can_post': True}}])
 list([])
 list([{'id': 19, 'from_id': 1700834, 'date': 1419319128, 'text': '"А, это тот, с голубыми куполами?" - говорят про наш храм', 'post_id': 1, 'owner_id': -81948990, 'parents_stack': [], 'thread': {'count': 0, 'items': [], 'can_post': True, 'show_reply_button': T

In [11]:
df['comment_count'] = df['comments'].apply(len)

In [13]:
df.head(15)

,inner_type,donut,is_pinned,comments,marked_as_ads,hash,type,attachments,copyright,date,...,post_type,reposts,text,views,copy_history,edited,carousel_offset,signer_id,zoom_text,comment_count
0,wall_wallpost,{'is_donut': False},1.0,[],0,NzCNvK0Y6J7nAD3X7w,post,"[{'type': 'photo', 'photo': {'album_id': -7, '...","{'link': 'https://vk.com/wall95551542_37647', ...",1728851421,...,post,{'count': 1},РАСПИСАНИЕ БОГОСЛУЖЕНИЙ В КРЕСТОВОЗДВИЖЕНСКОМ ...,{'count': 1261},NaN,NaN,NaN,NaN,NaN,0
1,wall_wallpost,{'is_donut': False},NaN,[],0,lggWRwObLrwQc82fRw,post,[],NaN,1729227601,...,post,{'count': 2},,{'count': 192},"[{'inner_type': 'wall_wallpost', 'type': 'post...",NaN,NaN,NaN,NaN,0
2,wall_wallpost,{'is_donut': False},NaN,[],0,HBq21UVQQx7GMRwuGQ,post,"[{'type': 'photo', 'photo': {'album_id': 27261...",NaN,1729172061,...,post,{'count': 5},"20 октября, в воскресенье, Божественную литург...",{'count': 994},NaN,NaN,NaN,NaN,NaN,0
3,wall_wallpost,{'is_donut': False},NaN,"[{'id': 13337, 'from_id': 132910578, 'date': 1...",0,BiC7JxDQasKuYe2ZuA,post,[],NaN,1729082164,...,post,{'count': 2},,{'count': 158},"[{'inner_type': 'wall_wallpost', 'type': 'post...",NaN,NaN,NaN,NaN,1
4,wall_wallpost,{'is_donut': False},NaN,[],0,TlHsjDIgikZL_4LoAA,post,"[{'type': 'photo', 'photo': {'album_id': -7, '...",NaN,1729031499,...,post,{'count': 5},"Огромное спасибо всем, кто уже поучаствовал в ...",{'count': 819},NaN,1.729066e+09,NaN,NaN,NaN,0
5,wall_wallpost,{'is_donut': False},NaN,[],0,4z6zFphTBuV-Cz4byA,post,[],NaN,1729004260,...,post,{'count': 1},,{'count': 848},"[{'inner_type': 'wall_wallpost', 'type': 'post...",NaN,NaN,NaN,NaN,0
6,wall_wallpost,{'is_donut': False},NaN,"[{'id': 13331, 'from_id': 107273503, 'date': 1...",0,zw_gxo4RS9HXzYknvw,post,"[{'type': 'video', 'video': {'response_type': ...",NaN,1728984200,...,post,{'count': 3},...В ЖЕНСКОМ ЕСТЕСТВЕ МУЖСКУЮ КРЕПОСТЬ ИМЕЛА Е...,{'count': 523},NaN,1.728984e+09,0.0,NaN,NaN,1
7,wall_wallpost,{'is_donut': False},NaN,[],0,at4CLVJ-lB9KocWx_g,post,"[{'type': 'photo', 'photo': {'album_id': -7, '...",NaN,1728899214,...,post,{'count': 8},О Всепетая Мати! О Всепетая Мати! \n \nПред ик...,{'count': 1426},NaN,NaN,0.0,NaN,NaN,0
8,wall_wallpost,{'is_donut': False},NaN,"[{'id': 13312, 'from_id': 3858045, 'date': 172...",0,7xCviXQO6chnynu9YQ,post,"[{'type': 'photo', 'photo': {'album_id': -7, '...",NaN,1728827456,...,post,{'count': 3},,{'count': 1011},"[{'inner_type': 'wall_wallpost', 'type': 'post...",1.728926e+09,NaN,NaN,NaN,10
9,wall_wallpost,{'is_donut': False},NaN,[],0,wg61QEt_2bYE4gLMZw,post,"[{'type': 'photo', 'photo': {'album_id': -7, '...",NaN,1728810720,...,post,{'count': 3},14 октября – Покров Пресвятой Богородицы.\nВ н...,{'count': 863},NaN,NaN,0.0,NaN,NaN,0


In [14]:
df['comments_text'] = df['comments'].apply(lambda x: ' '.join([comment['text'] for comment in x]))

In [15]:
df.head(10)

,inner_type,donut,is_pinned,comments,marked_as_ads,hash,type,attachments,copyright,date,...,reposts,text,views,copy_history,edited,carousel_offset,signer_id,zoom_text,comment_count,comments_text
0,wall_wallpost,{'is_donut': False},1.0,[],0,NzCNvK0Y6J7nAD3X7w,post,"[{'type': 'photo', 'photo': {'album_id': -7, '...","{'link': 'https://vk.com/wall95551542_37647', ...",1728851421,...,{'count': 1},РАСПИСАНИЕ БОГОСЛУЖЕНИЙ В КРЕСТОВОЗДВИЖЕНСКОМ ...,{'count': 1261},NaN,NaN,NaN,NaN,NaN,0,
1,wall_wallpost,{'is_donut': False},NaN,[],0,lggWRwObLrwQc82fRw,post,[],NaN,1729227601,...,{'count': 2},,{'count': 192},"[{'inner_type': 'wall_wallpost', 'type': 'post...",NaN,NaN,NaN,NaN,0,
2,wall_wallpost,{'is_donut': False},NaN,[],0,HBq21UVQQx7GMRwuGQ,post,"[{'type': 'photo', 'photo': {'album_id': 27261...",NaN,1729172061,...,{'count': 5},"20 октября, в воскресенье, Божественную литург...",{'count': 994},NaN,NaN,NaN,NaN,NaN,0,
3,wall_wallpost,{'is_donut': False},NaN,"[{'id': 13337, 'from_id': 132910578, 'date': 1...",0,BiC7JxDQasKuYe2ZuA,post,[],NaN,1729082164,...,{'count': 2},,{'count': 158},"[{'inner_type': 'wall_wallpost', 'type': 'post...",NaN,NaN,NaN,NaN,1,"Можно попробовать, сходить"
4,wall_wallpost,{'is_donut': False},NaN,[],0,TlHsjDIgikZL_4LoAA,post,"[{'type': 'photo', 'photo': {'album_id': -7, '...",NaN,1729031499,...,{'count': 5},"Огромное спасибо всем, кто уже поучаствовал в ...",{'count': 819},NaN,1.729066e+09,NaN,NaN,NaN,0,
5,wall_wallpost,{'is_donut': False},NaN,[],0,4z6zFphTBuV-Cz4byA,post,[],NaN,1729004260,...,{'count': 1},,{'count': 848},"[{'inner_type': 'wall_wallpost', 'type': 'post...",NaN,NaN,NaN,NaN,0,
6,wall_wallpost,{'is_donut': False},NaN,"[{'id': 13331, 'from_id': 107273503, 'date': 1...",0,zw_gxo4RS9HXzYknvw,post,"[{'type': 'video', 'video': {'response_type': ...",NaN,1728984200,...,{'count': 3},...В ЖЕНСКОМ ЕСТЕСТВЕ МУЖСКУЮ КРЕПОСТЬ ИМЕЛА Е...,{'count': 523},NaN,1.728984e+09,0.0,NaN,NaN,1,"Святая благоверная Анна Кашинская, моли Бога о..."
7,wall_wallpost,{'is_donut': False},NaN,[],0,at4CLVJ-lB9KocWx_g,post,"[{'type': 'photo', 'photo': {'album_id': -7, '...",NaN,1728899214,...,{'count': 8},О Всепетая Мати! О Всепетая Мати! \n \nПред ик...,{'count': 1426},NaN,NaN,0.0,NaN,NaN,0,
8,wall_wallpost,{'is_donut': False},NaN,"[{'id': 13312, 'from_id': 3858045, 'date': 172...",0,7xCviXQO6chnynu9YQ,post,"[{'type': 'photo', 'photo': {'album_id': -7, '...",NaN,1728827456,...,{'count': 3},,{'count': 1011},"[{'inner_type': 'wall_wallpost', 'type': 'post...",1.728926e+09,NaN,NaN,NaN,10,Царствие небесное 🙏🙏🙏 ❤🙏❤🙏 🙏🙏🙏 Царствие Небесн...
9,wall_wallpost,{'is_donut': False},NaN,[],0,wg61QEt_2bYE4gLMZw,post,"[{'type': 'photo', 'photo': {'album_id': -7, '...",NaN,1728810720,...,{'count': 3},14 октября – Покров Пресвятой Богородицы.\nВ н...,{'count': 863},NaN,NaN,0.0,NaN,NaN,0,


In [17]:
df['date'] = df['date'].apply(lambda x: datetime.fromtimestamp(x, tz=pytz.timezone('Europe/Moscow')))

In [19]:
result_df = df[['id','date','likes','reposts', 'views', 'comment_count', 'comments_text']]

In [20]:
result_df.head()

,id,date,likes,reposts,views,comment_count,comments_text
0,13325,2024-10-13 23:30:21+03:00,"{'can_like': 0, 'count': 43, 'user_likes': 0}",{'count': 1},{'count': 1261},0,
1,13340,2024-10-18 08:00:01+03:00,"{'can_like': 0, 'count': 10, 'user_likes': 0}",{'count': 2},{'count': 192},0,
2,13338,2024-10-17 16:34:21+03:00,"{'can_like': 0, 'count': 38, 'user_likes': 0}",{'count': 5},{'count': 994},0,
3,13333,2024-10-16 15:36:04+03:00,"{'can_like': 0, 'count': 5, 'user_likes': 0}",{'count': 2},{'count': 158},1,"Можно попробовать, сходить"
4,13332,2024-10-16 01:31:39+03:00,"{'can_like': 0, 'count': 21, 'user_likes': 0}",{'count': 5},{'count': 819},0,


In [21]:
result_df['likes'] = result_df['likes'].apply(lambda x: int(x['count']) if isinstance(x, dict) else 0)
result_df['reposts'] = result_df['reposts'].apply(lambda x: int(x['count']) if isinstance(x, dict) else 0)
result_df['views'] = result_df['views'].apply(lambda x: int(x['count']) if isinstance(x, dict) else 0)

<ipython-input-21-3b9fc7075914>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['likes'] = result_df['likes'].apply(lambda x: int(x['count']) if isinstance(x, dict) else 0)
<ipython-input-21-3b9fc7075914>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['reposts'] = result_df['reposts'].apply(lambda x: int(x['count']) if isinstance(x, dict) else 0)
<ipython-input-21-3b9fc7075914>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

In [22]:
result_df.head()

,id,date,likes,reposts,views,comment_count,comments_text
0,13325,2024-10-13 23:30:21+03:00,43,1,1261,0,
1,13340,2024-10-18 08:00:01+03:00,10,2,192,0,
2,13338,2024-10-17 16:34:21+03:00,38,5,994,0,
3,13333,2024-10-16 15:36:04+03:00,5,2,158,1,"Можно попробовать, сходить"
4,13332,2024-10-16 01:31:39+03:00,21,5,819,0,


In [23]:
result_df['date'] = result_df['date'].dt.strftime('%Y-%m-%d %H:%M:%S')

<ipython-input-23-0ebb9671c4d6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['date'] = result_df['date'].dt.strftime('%Y-%m-%d %H:%M:%S')


In [24]:
result_df.head()

,id,date,likes,reposts,views,comment_count,comments_text
0,13325,2024-10-13 23:30:21,43,1,1261,0,
1,13340,2024-10-18 08:00:01,10,2,192,0,
2,13338,2024-10-17 16:34:21,38,5,994,0,
3,13333,2024-10-16 15:36:04,5,2,158,1,"Можно попробовать, сходить"
4,13332,2024-10-16 01:31:39,21,5,819,0,


In [25]:
result_df.to_csv('krest_sobor_all_post_with_comments.csv', index = False)